In [2]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

import mlflow

In [40]:
experiment_id = mlflow.create_experiment(
    "MNIST MLP classifier", 
    artifact_location=Path.cwd().joinpath("./meta/mlflow/mlartifacts").as_uri(),
    tags={"version":"v1", "priority":"P1"}
)

In [41]:
experiment_id

'0'

In [42]:
experiment = mlflow.set_experiment("MNIST MLP classifier")

In [43]:
mlflow.set_tracking_uri("http://localhost:5000")

In [6]:
(train_images,train_labels),(test_images,test_labels) = mnist.load_data()
print(len(train_images))

60000


In [7]:
model = keras.Sequential([layers.Dense(512,activation='relu'),
layers.Dense(10,activation='softmax')])
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [8]:
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000,28*28))
test_images = test_images.astype("float32") / 255

In [44]:
mlflow.keras.autolog()
histroy = model.fit(train_images,train_labels,epochs = 5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 8.8811e-09 - accuracy: 1.0000
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 8.6248e-09 - accuracy: 1.0000
Epoch 3/5
469/469 [==============================] - 2s 3ms/step - loss: 8.2731e-09 - accuracy: 1.0000
Epoch 4/5
469/469 [==============================] - 1s 3ms/step - loss: 8.0943e-09 - accuracy: 1.0000
Epoch 5/5
469/469 [==============================] - 1s 3ms/step - loss: 7.8817e-09 - accuracy: 1.0000


In [59]:
with mlflow.start_run(experiment_id=experiment_id, description="parent") as run:
    mlflow.keras.log_model(model, "models")
    

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When f

In [60]:
run.info.run_id

'cb48122c54084acbbd40a8adeb971dce'

In [29]:
test_img = test_images[0:2][:]
test_labl = test_labels[0:2]
pred_labl = model.predict(test_img)

In [30]:
print(f"Predicted error {test_labl[0] - pred_labl[0].argmax()}") 

Predicted error 0


In [31]:
test_loss, test_accuracy = model.evaluate(test_img,test_labl)
print('Test Loss: ', test_loss)
print('Test Accuracy: ',test_accuracy*100)

1/1 [==============================] - 0s 91ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test Loss:  0.0
Test Accuracy:  100.0


In [32]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)
print('Test Loss: ', test_loss)
print('Test Accuracy: ',test_accuracy*100)

313/313 [==============================] - 1s 3ms/step - loss: 0.1494 - accuracy: 0.9837
Test Loss:  0.14935661852359772
Test Accuracy:  98.36999773979187


## Loading model for predictions

In [61]:
keras_model = mlflow.keras.load_model(f"runs:/{run.info.run_id}" + "/models")

In [62]:
predictions = keras_model.predict(test_img)

In [64]:
predictions[0].argmax()

7

In [88]:
test_images[0].shape

(784,)

## Infering with served model

In [94]:
import requests
import json
import numpy as np

In [ ]:
json_data = {'data': np.expand_dims(test_images[0], axis=0).tolist()}

In [96]:
headers = {'Content-type': 'application/json'}
res = requests.post('http://127.0.0.1:5000/invocations', headers=headers, data=json.dumps(json_data))
print(res.text)

[{"0": 4.766670946843021e-10, "1": 5.465215411303015e-13, "2": 9.226603481238271e-09, "3": 5.619980947813019e-05, "4": 3.853055600619776e-17, "5": 5.215898268851049e-10, "6": 6.385581625232239e-18, "7": 0.9999409914016724, "8": 6.213520808984185e-10, "9": 2.7281921575195156e-06}]
